<a href="https://colab.research.google.com/github/joseverajim/Actividad-3.1-3.2-y-3.3-Valores-Nulos-/blob/main/Actividad3.3/Actividad_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import pandas as pd
import numpy as np
df2023 = pd.read_csv('Ciudad_Pais.csv')
print(df2023.columns)
print(df2023.shape)

Index(['id', 'name', 'description', 'neighbourhood_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month', 'host_id', 'host_name',
       'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count',
       'host_identity_verified', 'instant_bookable',
       'calculated_host_listings_count',
       'calculate

In [138]:
#Revisar cuántos vacíos hay por columna
print(df2023.isnull().sum())

id                                                 0
name                                               0
description                                      361
neighbourhood_cleansed                             0
latitude                                           0
longitude                                          0
property_type                                      0
room_type                                          0
accommodates                                       0
bathrooms                                       2145
bedrooms                                         257
beds                                            2144
amenities                                          0
price                                           2138
minimum_nights                                     0
maximum_nights                                     0
availability_30                                    0
availability_60                                    0
availability_90                               

In [152]:
# --- Eliminación de la columna host_location ---
# La columna host_location se descarta porque presenta un número muy alto de valores nulos
# (más de la mitad del total de registros) y no aporta información consistente, además se
# había colado en la base generando 51 columnas cuando deberían ser 50.
df2023 = df2023.drop(columns=['host_location'], errors='ignore')

In [140]:
# Los valores nulos en description se reemplazaron por "No Description", ya que corresponden a anuncios sin información adicional del anfitrión.
df2023['description'] = df2023['description'].fillna("No Description")

In [141]:
# bathrooms, bedrooms, beds, price → mediana
# Usamos la MEDIANA porque:
# - Es robusta a valores extremos (ej: un Airbnb con 50 camas o un precio exagerado). Representa mejor el "valor típico" en este tipo de variables numéricas y así evitamos que la imputación se distorsione por outliers.
for col in ["bathrooms", "bedrooms", "beds", "price"]:
    if col in df2023.columns:
        df2023[col] = pd.to_numeric(df2023[col], errors="coerce")
        df2023[col] = df2023[col].fillna(df2023[col].median())

# amenities → texto
# Aquí no tiene sentido usar mediana ni promedio porque es una lista de características. Simplemente rellenamos con "[]" (lista vacía) para indicar que no se registraron amenities.
if "amenities" in df2023.columns:
    df2023["amenities"] = df2023["amenities"].fillna("[]")

In [142]:
#Se detecto que hay muchas columnas con un solo valor faltante, por lo que se procedio a inspeccionar si esta correspondia a la misma fila y en caso de ser asi eliminarla al tener tantos faltantes
# Seleccionamos solo las columnas con nulos
null_cols = df2023.columns[df2023.isna().any()]

# Filtramos filas que tengan al menos un nulo en esas columnas
rows_with_nulls = df2023[df2023[null_cols].isna().any(axis=1)]

print("Filas con nulos:", rows_with_nulls.shape[0])

# --- Ver si los nulos coinciden siempre en las mismas filas ---
# Contamos cuántas columnas están nulas en cada fila
rows_with_nulls["null_count"] = rows_with_nulls[null_cols].isna().sum(axis=1)

print(rows_with_nulls["null_count"].value_counts().head(10))

# --- Eliminamos esas filas ---
df2023_clean = df2023.dropna(subset=null_cols, how="all")  # elimina filas con todos nulos en esas cols

print("Shape original:", df2023.shape)
print("Shape limpio:", df2023_clean.shape)

Filas con nulos: 6226
null_count
1     2620
10    2601
11     348
4      191
2      159
3      142
14      80
12      42
13      32
5        6
Name: count, dtype: int64
Shape original: (22783, 50)
Shape limpio: (22783, 50)


/tmp/ipython-input-3120525580.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_with_nulls["null_count"] = rows_with_nulls[null_cols].isna().sum(axis=1)


In [143]:
# --- Imputación con PROMEDIO en variables de reseñas --- se rellenan los nulos con el promedio de cada columna para asignar un valor representativo del conjunto y mantener consistencia sin dejar huecos
# reviews_per_month
df2023['reviews_per_month'] = pd.to_numeric(df2023['reviews_per_month'], errors='coerce')
df2023['reviews_per_month'] = df2023['reviews_per_month'].fillna(df2023['reviews_per_month'].mean())

# review_scores_* (todas las métricas de reseñas)
score_cols = [c for c in df2023.columns if c.startswith('review_scores_')]
for c in score_cols:
    df2023[c] = pd.to_numeric(df2023[c], errors='coerce')
    df2023[c] = df2023[c].fillna(df2023[c].mean())

In [144]:
# --- Imputación en columnas first_review y last_review --- Estas columnas muestran la fecha de la primera y la última reseña; los nulos significan que el alojamiento nunca recibió reseñas, pero para mantener consistencia se sustituyen con la mediana temporal de cada columna, que representa el punto medio de las fechas observadas, y además se crea la columna no_review_dates para marcar con 1 los casos sin reseñas y con 0 los que sí tienen, de modo que no se pierde la referencia de cuáles registros fueron imputados.
for col in ['first_review','last_review']:
    df2023[f'was_na__{col}'] = df2023[col].isna().astype(int)
    df2023[col] = pd.to_datetime(df2023[col], errors='coerce')
    med = df2023[col].median()
    df2023[col] = df2023[col].fillna(med)

df2023['no_review_dates'] = df2023['first_review'].isna().astype(int)

In [145]:
#Revisar cuántos vacíos hay por columna
print(df2023.isnull().sum())

id                                                 0
name                                               0
description                                        0
neighbourhood_cleansed                             0
latitude                                           0
longitude                                          0
property_type                                      0
room_type                                          0
accommodates                                       0
bathrooms                                          0
bedrooms                                           0
beds                                               0
amenities                                          0
price                                              0
minimum_nights                                     0
maximum_nights                                     0
availability_30                                    0
availability_60                                    0
availability_90                               

In [146]:
# --- Imputación con promedio en estimated_revenue_l365d --- Se imputan los valores nulos con el promedio de la columna porque este valor representa al conjunto de datos y permite conservar la escala original, de esta forma se evita eliminar registros que podrían ser útiles, se mantiene la coherencia en la distribución general y no se introducen valores extremos que distorsionen el análisis.
df2023['was_na__estimated_revenue_l365d'] = df2023['estimated_revenue_l365d'].isna().astype(int)
df2023['estimated_revenue_l365d'] = df2023['estimated_revenue_l365d'].fillna(df2023['estimated_revenue_l365d'].mean())


In [147]:

import numpy as np

# --- Imputación de variables de host --- En host_is_superhost se usó la moda porque es una variable binaria y lo correcto es asignar el valor más frecuente; en host_response_time también se utilizó la moda ya que se trata de categorías de tiempo de respuesta y lo más adecuado es rellenar con la categoría más común; en host_response_rate y host_acceptance_rate se quitaron los símbolos de porcentaje y se imputaron con el promedio ya que son variables numéricas continuas y el promedio permite darles un valor representativo del conjunto sin alterar las categorías existentes.

# host_is_superhost
df2023['was_na__host_is_superhost'] = df2023['host_is_superhost'].isna().astype(int)
df2023['host_is_superhost'] = df2023['host_is_superhost'].fillna(df2023['host_is_superhost'].mode()[0])

# host_response_time
df2023['was_na__host_response_time'] = df2023['host_response_time'].isna().astype(int)
df2023['host_response_time'] = df2023['host_response_time'].fillna(df2023['host_response_time'].mode()[0])

# host_response_rate y host_acceptance_rate
for col in ['host_response_rate','host_acceptance_rate']:
    df2023['was_na__' + col] = df2023[col].isna().astype(int)
    df2023[col] = df2023[col].astype(str).str.replace('%','').replace('nan', np.nan).astype(float)
    df2023[col] = df2023[col].fillna(df2023[col].mean())



In [148]:
nulos = df2023.isnull().sum().reset_index()
nulos.columns = ['columna','nulos']
print(nulos.sort_values(by='nulos', ascending=False))

                                         columna  nulos
0                                             id      0
1                                           name      0
2                                    description      0
3                         neighbourhood_cleansed      0
4                                       latitude      0
5                                      longitude      0
6                                  property_type      0
7                                      room_type      0
8                                   accommodates      0
9                                      bathrooms      0
10                                      bedrooms      0
11                                          beds      0
12                                     amenities      0
13                                         price      0
14                                minimum_nights      0
15                                maximum_nights      0
16                               availability_30

In [160]:
# --- Volver a EXACTAMENTE las columnas originales del CSV ---
orig_cols = list(pd.read_csv("Ciudad_Pais.csv", nrows=0).columns)  # columnas originales (deben ser 50)


# --- Eliminación de la columna host_location ---
# La columna host_location se descarta porque presenta un número muy alto de valores nulos
# (más de la mitad del total de registros) y no aporta información consistente, además se
# había colado en la base generando 51 columnas cuando deberían ser 50.
df2023 = df2023.drop(columns=['host_location'], errors='ignore')

# Reportar qué columnas sobran
extras = [c for c in df2023.columns if c not in orig_cols]
print("Columnas extra que se eliminarán:", extras)

# Mantener solo las originales, en el mismo orden
df2023 = df2023.reindex(columns=orig_cols)

Columnas extra que se eliminarán: []


In [161]:
# --- Eliminación de la columna host_location ---
# La columna host_location se descarta porque presenta un número muy alto de valores nulos
# (más de la mitad del total de registros) y no aporta información consistente, además se
# había colado en la base generando 51 columnas cuando deberían ser 50.
df2023 = df2023.drop(columns=['host_location'], errors='ignore')

In [162]:
# --- Verificación de nulos en la baseaFrame:"
(df2023.isna().sum().sum())

# Conteo de nulos por columna
print("\nNulos por columna:")
print(df2023.isna().sum())
print(df2023.shape)


Nulos por columna:
id                                              0
name                                            0
description                                     0
neighbourhood_cleansed                          0
latitude                                        0
longitude                                       0
property_type                                   0
room_type                                       0
accommodates                                    0
bathrooms                                       0
bedrooms                                        0
beds                                            0
amenities                                       0
price                                           0
minimum_nights                                  0
maximum_nights                                  0
availability_30                                 0
availability_60                                 0
availability_90                                 0
availability_365              

In [163]:
# --- Guardar la base limpia con nombre Tokyo ---
df2023.to_csv("Tokyo_Limpia.csv", index=False, encoding="utf-8")